# Welcome to BME590, Python, and Jupyter!




First thing we need to do: get you up to speed on writing some Python and using Jupyter Notebook. You'll turn in your homeworks as .ipynb Jupyter pages. (You have almost two weeks to come up to speed before turning in your first one.)

You can use the Python language in different ways: interactively one line at a time, or by writing Python programs in files and executing them, or inside a Jupyter notebook page. Jupyter is convenient for trying code out, and for playing with data visualization. Some people even use Jupyter (or systems like it) as an electronic lab notebook. Personally, I use Jupyter for playing around and prototyping. For permanent stuff, I move working code out to a `.py` file. You'll want to learn how to use a text editor for writing Python code, in addition to working within Jupyter. We'll just worry about Jupyter right now though.

A page is a mix of code and text blocks, interleaved. Each code block can be "run" (executed) separately, or all at once. This right now is a text block.

Text is written in Markdown, a super simple "language" that allows basic formatting of plain text. You can make stuff **bold** or _italic_ and so on while still writing a text file.

Code is written, well, as code. You'll see below.

(Normally your Jupyter page wouldn't have as much explanatory text as I'm about to write! Likewise, the code is going to look longer than it needs to be, because it's loaded up with comments)

You might want to hit the `Run All` button in Jupyter now (look under the `Cell` menu), to run all the code in this page. Or, you can run each cell as you reach it with `Shift-Return`. If you double-click on the text cells, you'll see the Markdown text behind the pretty formatted stuff, and you'll see by example how to write math symbols, bold and italic, and so on.


### A starting example: our first sand mouse experiment

Sean Eddy tells this story about how he was reviewing a grant proposal on mouse behavior. The scientists were measuring the average time it takes a mouse to complete a maze. They have mutant mice and control (normal) mice, and they want to decide whether the mutant mice run the maze faster or slower than normal. Because there's natural variation, they have to collect times from several mutant and several control mice before they can conclude that mutant mice are faster or slower on average.

That is: suppose I only do _one_ mouse each. Mutant mouse X1 runs in 10.3 seconds; control mouse C1 runs in 11.1 seconds. Is the mutant mouse faster? Well, sure, in a trivial sense; X1 won. But how about in the sense of _predicting what happens in the next race_? Maybe all these mice are statistically the same, running the maze in 10.7 $\pm$ 1.0 seconds.

To make predictions, I need (at least) to know how variable the times are, as I test new mice. Suppose I time a thousand mutant and control mice; I might see a beautiful bell curve for each distribution. If the two distributions are well separated, then I'm sure that the mice have significantly different speeds. The more the two distributions overlap, the less sure I am that the mice are significantly different.

Nobody wants to be doing experiments on thousands of mice. So: how many mice should I use? I want to use enough mice that I can see a significant effect if the effect is present, while being able to distinguish the effect (_between samples_) from the other variability (_within samples_).

Of course, to figure this out (even as a back of the envelope calculation) you're going to have to state some assumptions. Typical assumptions would be, let's assume that the two distributions are bell-shaped ('normal', Gaussian). Let's assume that we're trying to distinguish whether they have _different_ means versus the _same_ mean, but they have the same variance (width; standard deviation). That is: control mouse times come in a Gaussian distribution with mean $\mu_0$ and variance $\sigma^2$; mutant mouse times have the same variance and mean $\mu_1$; and we're trying to decide if $\mu_0 = \mu_1$ or not.

Of course our problem is that we don't know the true values $\mu_0$ and $\mu_1$ (or we'd be done). We can only estimate them by averaging over a bunch of mice we measure. The more mice we measure, the more accurately we can estimate $\mu_0$ and $\mu_1$. Let's call the observed average times $\bar{x}_0$ and $\bar{x}_1$. For any finite sample size (especially small samples), we're going to observe a difference $| \bar{x}_0 - \bar{x}_1 | > 0$ even if $\mu_0 = \mu_1$. We'll have to do some sort of statistical test to decide whether the observed difference is "significant" or not.

We'll get into significance testing soon in the course. For now, what you need to know (or remember, from stats you've studied) is that we care about two kinds of probabilities:

   * what's the probability that we observe a difference above some threshold even though $\mu_0 = \mu_1$, just because of statistical fluctuation, such that we incorrectly conclude that the mice are different? (These are _false positives_.)
   
   * what's the probability that we _don't_ observe a difference above some threshold when $\mu_0 \neq \mu_1$, and we incorrectly conclude that the mice are the same? (These are _false negatives_.)
   
If you think about it, you'll realize that we're also going to have to state one more thing, which is how big the mutant effect is -- how different $\mu_0$ and $\mu_1$ are -- to address the second probability. If $\mu_0$ and $\mu_1$ are really different, it'll only take a few animals to see. If they're super close to each other, it'll take a very accurate estimate to see the difference, hence lots of animals.

We can't literally drive these two error probabilities to zero, unless we use an infinite number of animals, so we have to make vexing choices about what error probabilities we're willing to tolerate.

In the grant Sean was reading, the authors said they were going to use n=8 control mice and n=8 mutant mice. They said they wanted to detect an effect that shifted $\mu_1$ by half a standard deviation (0.5 $\sigma$). They said that they would set a difference threshold at a false positive rate of 0.05. They said that they'd use a standard statistical test called the two-tailed t-test to decide whether the two means were significantly different or not. Under these experimental conditions, they claimed that they would be able to detect the effect with a probability of 80% (i.e. a false negative probability of $\leq$ 20%).

Sean looked at this and thought, no frickin' way. Only 8 animals, and you're doing to detect an effect that's only half the width of the variance, two bell-shaped distributions right on top of each other?

To really assess the validty of the grant's claim, you'd have to know what a two-tailed t-est was. That the $\mu$ and $\sigma$ are called _population parameters_, the unknown true parameters, and the data we observe are _sample means_ $\bar{x}$ and _sample variances_ $s^2$. The false positive probability is called the _p-value_. One minus the false negative probability is the _statistical power_. But we don't need to know all this -- and indeed, I'm going to argue throughout the course, it's better _not_ to know it at first, and instead to approach problems experimentally by simulation. For instance, I think it's confusing to try to remember the difference between a population mean and a sample mean, but it's non-confusing to simulate data where you will naturally have to distinguish between the true parameter that you simulated the data with, versus a parameter you've estimated from the simulated data.

So let's see whether the authors are right about their n=8 being enough, by brute force simulation in Python. Let's write a Python function that simulates one experiment, and then let's run 10,000 experiments (because we can, it's just a computer's time we're burning), and see how often we get false positives and false negatives.

In [62]:
# This is Python code now.
#
# Anything after a `#` is a comment, where you can write anything you want and Python ignores it.
#
# The first thing we do is `import` any code modules we're going to use, and we can assign shorthand
# names to them. `import numpy as np` means we want to use the standard `numpy` module, Numerical Python,
# and when we call NumPy routines we're going to call them `np.something`.
#
import numpy as np                 # NumPy: numerical python. Includes random sampling from a Gaussian.
import scipy.stats as stats         # SciPy: scientific python. Its `stats` module includes standard stats tests like the t-test.
import matplotlib.pyplot as plt     # matplotlib: graphing and data visualization. Which we won't use today!

In [63]:
# In Jupyter, lines that start with % aren't Python code, they are special Jupyter commands.
# These lines would throw syntax errors if you were writing Python alone in a file.
# A special command we're going to use almost every week in the course is to put the
# matplotlib module into a Jupyter notebook mode, for interactive, in-line visualizations.
# That command is:

%matplotlib notebook

# We're not using it here, but it's going to be something we standardly do in all Jupyter pages in the course.

In [64]:
# To assign a variable to a value, the Python syntax is just <variable_name> = <value>.
#
# In your scripts, it's generally good form to assign all your tunable parameters at
# the beginning, and document them clearly.
#
# Then you can explore easily, changing parameters and re-running the page.

# ----
# Which variables to we need to define to run our simulation experiment?
N = 70 # number of mutant and control mice (each group)
mu0 = 0
mu1 = 0.5
sigma = 1

n_tot = 10000 # Total numbero f experiments to run
n_sig = 0# Counter for how many experiments are significant. Initialize it to zero

In [65]:
# To define a function, the Python syntax is
#    def <functionname>(<arguments>):
#        <code>
#
# Python is sensitive to indentation, unlike other languages that
# use special symbols for end-of-line or to delimit blocks of code. Lines
# with the same indentation belong to the same "block". This will become
# more important when we're seeing control structures like if/then.
#
# You may know that Python has a special syntax for writing function
# documentation - but we're not going to use it (yet), so we can keep
# things bare-bones.

# experiment() runs one complete mouse experiment, for known parameters,
# and returns the "significance" of the difference between the control
# and mutant mouse samples: the p-value according to a two-tailed t-test.
#
np.random.seed(1)
def experiment(mu0, mu1, sigma, N): # What parameters should experiment() take in to run a single simulation?
    # How do we simulate N control and N mutant mouse times?
    # Let's say x0 will be a vector (or array) of N observations for the control animals,
    # and x1 will be a vector of N observations for the mutant animals
    x0 = np.random.normal(mu0, sigma, N)
    x1 = np.random.normal(mu1, sigma, N)

    # Calculate the significance (p-value) of the difference between means of the two samples.
    # We'll worry later in the course about what a two-tailed t-test is actually doing.
    # For now, all we're doing is testing the authors' assertions about the false positive
    # and false negative rate of the t-test for n=8 mice.
    #
    t_stat, pvalue = stats.ttest_ind(x0, x1)     # This is a call to the t-test in SciPy's stats module.
                                                 # It returns what Python calls a `tuple`: an ordered list of things.
                                                 # Here, that's a tuple of two numbers: the t-test statistic, and its p-value.

    # What the heck is a t statistic and a t-test, you wonder?
    # Oh, ok, let's peek a little bit inside the hood. We don't need this code for what we're doing today but...

    # We need to calculate the mean and variance for x0 and x1. How?
    mean0 = np.mean(x0)  # np.mean() just calculates the mean (arithmetic average) of a vector, and returns it.
    mean1 = np.mean(x1)

    var0  = np.var(x0, ddof=1)  # and np.var() just calculates the variance.
    var1  = np.var(x1, ddof=1)  # `ddof=1` is an example of passing an optional argument to a function by name=value,
                                # rather than simply by order. ddof is "degrees of freedom", a technicality in the
                                # t-test calculation that we'll ignore for now, and get into later.

    # Here's the eqn for a t-test statistic. SciPy's stats.ttest_ind function calculated this,
    # then it calculated how often you'd observe a t-statistic at least this
    # large, if the two means were equal: that's the "p-value".
    #
    t_mine = (x0 - x1) / np.sqrt( var0 / N + var1 / N)

    # How do you print stuff in Python?
    # It's as simple as
    #    print("hello world!")
    # but if you want to stick variables into what you print, you do something like:
    #    print("value 1 = {0}   and    value 2 = {1}".format(v1, v2))
    # and if you want to be super fancy and format your output variables to certain
    # width and precision so tables look organized, something like {0:6.2f} means "format
    # variable 0 as a 6-wide, 2-digits-after-the-decimal, floating point number".
    #

    # -----
    # Knowing this, let's write a line that will print out relevant variables for each run of experiment()
    # ...
    # ...
   # print("{0:6.2f}  {1:6.2f}  {2:7.4f}  {3:7.4f}  {4:6.4f}".format(mean0, mean1, t_stat, t_mine[0], pvalue))


    # To get a result back from a function, you 'return' it: either a single value, or a tuple.
    return pvalue

### Negative control

In our negative control, we simulate 10,000 experiments where there's no difference: both the control and mutant mice have the same mean $\mu_0$. The authors said they choose a p-value threshold of 0.05, which means that they're going to tolerate a 5% false positive probability. This literally means that 5% of the time, you observe a "significant" difference, even though there is no real difference, just because of statistical fluctuation. If we're using the t-test properly, 5% of the 10,000 negative control experiments should be "significant":

In [66]:
# A `for` loop iterates the same code block.
# `range(n_tot)` produces an array of every integer from 0 to n_tot-1.
# `for i in range(n_tot)` means run the code block with i=0, i=1, ..., i=n_tot-1.
# That is, run it 10,000 times if n_tot=10,000.

# ----
# Now let's write a loop to run our experiment n_tot times and see whether we have
# 5% false positives when the means are equal.
# ----
n_sig = 0
for i in range(n_tot):
  pvalue = experiment(mu0, mu0, sigma, N)
  if pvalue <= 0.05:
    n_sig += 1

and is it 5%?

In [67]:
print("Number of significant results = {0}/{1} ({2:.3f})".format(n_sig, n_tot, n_sig/n_tot))

Number of significant results = 528/10000 (0.053)


### Positive control

Or call it our experiment, if you prefer: now we simulate 10,000 experiments where the control and mutant mice really do have different means $\mu_0$ and $\mu_1$, and ask how often we detect a "significant" result. The authors claimed this would be about 80%. Seems like BS to me. Is it? Same code, without the comments, and now calling `experiment()` with different true means:

In [68]:
# ----
# Now let's write a loop to run our experiment n_tot times and see whether we have
# 5% false positives or not.
# ----
n_sig = 0
for i in range(n_tot):
    pvalue = experiment(mu0, mu1, sigma, N)
    if pvalue <= 0.05:
        n_sig += 1

and is it 80%?...

In [69]:
print("Number of significant results = {0}/{1} ({2:.3f})".format(n_sig, n_tot, n_sig/n_tot))

Number of significant results = 8347/10000 (0.835)


yeah... well... um.

With n=8 animals, the authors are nowhere near being able to reliably detect an effect as small as what they say they want to detect. They're barely getting more significant results than they expect just by chance.

If you treat statistics as black box recipes, it's way too easy to make silly mistakes. We're going to learn in the course how to do simple simulations as control experiments to double check that you're not making silly errors.

## stuff you can do

How many mice do they really need to achieve an 80% probability of detecting a "significant" result when there's really a difference? Change the parameters and re-run the page, iteratively, until you narrow in on an answer.

Conversely, if they're really only going to use n=8 control and mutant mice, how big does the effect size have to be, for them to detect it 80% of the time?


## stuff you can think about

Is it ok to do science experiments where you have a 5% chance of reporting an effect, when no effect is present?

Is it ok to do science experiments where you have a 20% chance of missing an effect that's really there?